In [ ]:
import SuperSCC as scc
from os import mkdir, chdir, getcwd
from os.path import exists

In [2]:
# get file loc 
file_loc = pd.read_csv("/mnt/disk5/zhongmin/superscc/结果位置/结果位置_3.csv")

file_loc = pd.DataFrame(file_loc.loc[file_loc.数据集.isin(['Banovich_Kropski_2020', 'Barbry_Leroy_2020', 'Krasnow_2020', 'Lafyatis_Rojas_2019', 'Meyer_2019', 'Misharin_2021', 'Misharin_Budinger_2018', 'Nawijn_2021', 'Teichmann_Meyer_2019']), ["数据集", "取到的2w子集文件的位置"]])

raw_cell_type_file_loc = list_files(path = "/home/fengtang/jupyter_notebooks/working_script/evulate_clustering/cell_type_info/raw", pattern=".+csv$", full_name=True)

finest_cell_type_file_loc = list_files(path = "/home/fengtang/jupyter_notebooks/working_script/evulate_clustering/cell_type_info/finest", pattern=".+finest.+csv$", full_name=True)

file_loc.loc[:, "raw_cell_type"] = [raw_cell_type_file_loc[2], raw_cell_type_file_loc[4], raw_cell_type_file_loc[6], raw_cell_type_file_loc[0], raw_cell_type_file_loc[5], raw_cell_type_file_loc[-2], raw_cell_type_file_loc[1], raw_cell_type_file_loc[-1], raw_cell_type_file_loc[3]]
file_loc.loc[:, "finest_cell_type"] = [finest_cell_type_file_loc[-2], finest_cell_type_file_loc[3], finest_cell_type_file_loc[-3], finest_cell_type_file_loc[1], finest_cell_type_file_loc[0], finest_cell_type_file_loc[-4], finest_cell_type_file_loc[2], finest_cell_type_file_loc[-1], finest_cell_type_file_loc[5]]

In [ ]:
def main(data, data2, filename, wk_dir, query_data = None, train_on = "single"):

    # read data in
    print(f"{scc.record_time()} read expression matrix in")
    if train_on == "single":
        data = pd.read_csv(data, index_col=0)
    else:
        data = data

    # specify the filename
    filename = filename 

    # read cell label in
    print(f"{scc.record_time()} read cell type information in")
    if train_on == "single":
        label = pd.read_csv(data2, index_col=0)
    else:
        label = data2

    # replace NA values 
    label = label.replace(np.nan, "Unknown")

    # create log_file object
    my_logger = log_file("model_training_and_prediction", "a")

    if train_on == "single":
        # split train dataset and test dataset index
        print(f"{scc.record_time()} split dataset for training and testing")
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(data, label, test_size= 0.3)

        # write train dataset and test dataset
        Xtrain.loc[:, "cell_type"] = Ytrain.cell_type.values
        Xtrain_index = pd.DataFrame(Xtrain.loc[:, "cell_type"])

        Xtest.loc[:, "cell_type"] = Ytest.cell_type.values
        Xtest_index = pd.DataFrame(Xtest.loc[:, "cell_type"])

        Xtrain_index.to_csv(f"{filename}_train_dataset.csv")
        Xtest_index.to_csv(f"{filename}_test_dataset.csv")
    else:
        Xtrain = data

    # do normalization on train dataset
    if train_on == "single":
        print(f"{scc.record_time()} doing normalization on training dataset")
        Xtrain = sc.AnnData(Xtrain.select_dtypes("number"))
        sc.pp.normalize_total(Xtrain, target_sum = 1e4)
        sc.pp.log1p(Xtrain)

        Xtrain2 = pd.DataFrame(Xtrain.X)
        Xtrain2.columns = Xtrain.var_names
        Xtrain2.index = Xtrain.obs_names
        Xtrain2.loc[:, "cell_type"] = Ytrain.cell_type.values
    else:
        print(f"{scc.record_time()} doing normalization on training dataset")
        Xtrain = sc.AnnData(Xtrain.select_dtypes("number"))
        sc.pp.normalize_total(Xtrain, target_sum = 1e4)
        sc.pp.log1p(Xtrain)

        Xtrain2 = pd.DataFrame(Xtrain.X)
        Xtrain2.columns = Xtrain.var_names
        Xtrain2.index = Xtrain.obs_names
        Xtrain2.loc[:, "cell_type"] = data2.cell_type.values

    if train_on == "single":
        # do normalization on test dataset
        print(f"{scc.record_time()} doing normalization on testing dataset")
        Xtest= sc.AnnData(Xtest.select_dtypes("number"))
        sc.pp.normalize_total(Xtest, target_sum = 1e4)
        sc.pp.log1p(Xtest)

        Xtest2 = pd.DataFrame(Xtest.X)
        Xtest2.columns = Xtest.var_names
        Xtest2.index = Xtest.obs_names
        Xtest2.loc[:, "cell_type"] = Ytest.cell_type.values
    else:
        Xtest = query_data
        print(f"{scc.record_time()} doing normalization on testing dataset")
        Xtest= sc.AnnData(Xtest.select_dtypes("number"))
        sc.pp.normalize_total(Xtest, target_sum = 1e4)
        sc.pp.log1p(Xtest)

        Xtest2 = pd.DataFrame(Xtest.X)
        Xtest2.columns = Xtest.var_names
        Xtest2.index = Xtest.obs_names
        Xtest2.loc[:, "cell_type"] = query_data.cell_type.values

    

    # run feature_selection
    print(f"{scc.record_time()} run feature selection")
    res1 = scc.feature_selection(Xtrain2.copy(), label_column = "cell_type", model = "svm", filename = filename, normalization_method = "Min-Max", logger = my_logger, save = True)
    features = [i[0] for i in res1["final_feature_selection_by_ensemble"]]

    # run model training
    print(f"{scc.record_time()} run model training")
    res2 = scc.model_training(Xtrain2.copy(), label_column = "cell_type", features = features, model = "svm", normalization_method = "Min-Max", filename = filename, save = True, logger = my_logger)

    # run prediction
    print(f"{scc.record_time()} run prediction")
    pred = scc.predict_label(Xtest2, models = ".+training_model.+pkl$", wk_dir = wk_dir, save=True, logger = my_logger)


In [ ]:
# self-validation
# train on 70% data of a dataset, predict on 30% data of the corresponding dataset

data_loc = file_loc.iloc[:, 1].values
data2_loc =  file_loc.iloc[:, 3].values
filenames = file_loc.iloc[:, 0].values

os.chdir("/mnt/disk5/zhongmin/superscc/label_transfer/代码/SuperSCC/")
for i in filenames:
    if not exists(i):
        mkdir("./" + i)


for i in range(len(filenames)):
    os.chdir(filenames[i])
    wk_dir = os.getcwd()
    main(data = data_loc[i], data2 = data2_loc[i], filename = filenames[i], wk_dir = wk_dir)
    print(wk_dir)
    os.chdir("/mnt/disk5/zhongmin/superscc/label_transfer/代码")

In [ ]:
# cross-validation
# train on one individual datasets 
# then predict on another dataset
cell_index = pd.read_csv("/home/fengtang/jupyter_notebooks/working_script/evulate_clustering/cell_index_from_each_study.csv", index_col=0)
cell_type = pd.read_csv("/mnt/disk5/zhongmin/superscc/师兄整理的肺数据/未去批次效应数据/没有去除批次效应_所有细胞分2万metadata.csv", index_col = 0)

count_matrix = pd.read_csv('/mnt/disk5/zhongmin/superscc/师兄整理的肺数据/未去批次效应couns数据/没有去除批次效应_所有细胞分2万.csv', index_col=0)
train_cell_index = cell_index.loc[cell_index.study.isin(["Misharin_Budinger_2018", "Misharin_2021", "Meyer_2019", "Seibold_2020", "Jain_Misharin_2021"])]

sub_count = count_matrix.loc[count_matrix.index.isin(train_cell_index.index)]
sub_cell_type = pd.DataFrame(cell_type.loc[cell_type.index.isin(train_cell_index.index), "ann_finest_level"])
sub_cell_type.columns = ["cell_type"]

sub_count.loc[:, "cell_type"] = sub_cell_type.cell_type.values

# prepare query dataset
query_file_loc = file_loc.loc[file_loc.数据集.isin(["Banovich_Kropski_2020", "Barbry_Leroy_2020", "Krasnow_2020", "Lafyatis_Rojas_2019", "Nawijn_2021", "Teichmann_Meyer_2019"])]

parent_dir = "/home/fengtang/jupyter_notebooks/working_script/label_transfer/SuperSCC"
os.chdir(parent_dir)

    
for i in range(query_file_loc.shape[0]):
    query_matrix = pd.read_csv(query_file_loc.iloc[i, 1], index_col=0)
    query_label = pd.read_csv(query_file_loc.iloc[i, 3], index_col=0)
    query_matrix.loc[:, "cell_type"] = query_label.cell_type.values
    filename = query_file_loc.iloc[i, 0]

    if not exists(filename):
        mkdir("./" + filename)
    
    os.chdir(filename)

    main(data = sub_count, data2 = sub_cell_type, filename = filename, query_data = query_matrix, wk_dir = getcwd(), train_on = "multi")

    os.chdir(parent_dir)

/tmp/ipykernel_451058/137759856.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_count.loc[:, "cell_type"] = sub_cell_type.cell_type.values
